In [12]:
import pandas as pd
from sklearn.model_selection import StratifiedShuffleSplit
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report



In [2]:
game_data = pd.read_csv("../Data/Processed_Data/game_data.csv")

In [3]:
game_stats = game_data[game_data.columns[2:26]]

In [4]:
game_stats.drop(["fourth_down_conversion_attempts", "first_downs", "turnovers", "completions", "total_yards", "rushing_attempts",
                "penalties_number", "sacks_number"], axis = 1, inplace = True)
dropped = ["fourth_down_conversion_attempts", "first_downs", "turnovers", "completions", "total_yards", "rushing_attempts",
                "penalties_number", "sacks_number"]

<ipython-input-4-d4015e979eaf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  game_stats.drop(["fourth_down_conversion_attempts", "first_downs", "turnovers", "completions", "total_yards", "rushing_attempts",


In [5]:
game_stats

,allowed,def_st_td,drives,fumbles,int,passing_yards,possession,rushing_yards,score,attempts,fourth_down_conversions,penalty_yards,sack_yards,third_down_conversions,third_down_attempts,Superbowl Status
0,389,3,252,8,11,3813,547.733333,2334,406,616,9,745,119,133,256,0
1,379,2,276,17,18,3051,449.016667,1344,281,543,10,864,265,71,210,0
2,456,7,268,13,22,3476,462.533333,1730,279,591,11,884,256,87,223,0
3,397,5,254,16,15,3995,501.250000,1596,379,612,7,1031,369,90,212,0
4,344,6,235,7,20,3824,498.266667,2266,392,554,16,877,315,77,202,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,401,2,187,11,12,3934,493.416667,2042,407,573,12,854,348,76,201,0
668,346,1,180,5,14,3411,535.883333,2073,308,571,11,711,250,106,236,0
669,420,1,198,9,19,3344,490.200000,1476,289,579,13,644,298,67,221,0
670,359,1,191,12,11,2914,501.483333,2131,298,456,9,957,313,80,219,0


In [6]:
split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.2, random_state = 69)

for train_index, test_index in split.split(game_stats, game_stats["Superbowl Status"]):
    game_train = game_stats.loc[train_index]
    game_test = game_stats.loc[test_index]

    

sb_train = game_train["Superbowl Status"]
sb_test = game_test["Superbowl Status"]

game_train = game_train.drop(["Superbowl Status"], axis = 1)
game_test = game_test.drop(["Superbowl Status"], axis = 1)

In [7]:
smote = SMOTE(random_state = 69)
game_train, sb_train = smote.fit_resample(game_train, sb_train)

In [22]:
#three components can capture 98 percent of variation, that's pretty good
pca = PCA(n_components = 2, whiten = True)

pca = pca.fit(game_train)

pca_game_train = pca.transform(game_train)
pca_game_test = pca.transform(game_test)

print(np.sum(pca.explained_variance_ratio_))

0.9425425866448531


In [23]:
grid = { 
        'C':[1, 10, 100, 1000],
        'penalty':['elasticnet', 'l1', 'l2'],
        'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

grid_svc = GridSearchCV(LogisticRegression(), grid, scoring = "recall_macro")

grid_svc = grid_svc.fit(pca_game_train, sb_train)
print(grid_svc.best_params_)

{'C': 1, 'penalty': 'l2', 'solver': 'sag'}


/home/chidurah/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:425: FitFailedWarning: 
160 fits failed out of a total of 300.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "/home/chidurah/.local/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/chidurah/.local/lib/python3.8/site-packages/sklearn/base.py", line 1152, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/home/chidurah/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py", line 1169, in fit
    solver = _check_solver(self.so

In [24]:
preds = grid_svc.predict(pca_game_test)
print(classification_report(sb_test, preds))
print(confusion_matrix(sb_test, preds))

              precision    recall  f1-score   support

           0       0.99      0.56      0.72       131
           1       0.05      0.75      0.09         4

    accuracy                           0.57       135
   macro avg       0.52      0.66      0.41       135
weighted avg       0.96      0.57      0.70       135

[[74 57]
 [ 1  3]]
